# 🔄 使用 GitHub 模型构建基础代理工作流 (.NET)

## 📋 工作流编排教程

本教程展示了如何使用 Microsoft Agent Framework for .NET 和 GitHub 模型构建复杂的**代理工作流**。您将学习如何创建多步骤的业务流程，让 AI 代理通过结构化的编排模式协作完成复杂任务。

## 🎯 学习目标

### 🏗️ **工作流架构基础**
- **工作流构建器**：设计并编排复杂的多步骤 AI 流程
- **代理协调**：在工作流中协调多个专业代理
- **GitHub 模型集成**：在工作流中利用 GitHub 的 AI 模型推理服务
- **可视化工作流设计**：创建并可视化工作流结构以便更好地理解

### 🔄 **流程编排模式**
- **顺序处理**：按逻辑顺序链接多个代理任务
- **状态管理**：在工作流阶段之间维护上下文和数据流
- **错误处理**：实现强大的错误恢复和工作流弹性
- **性能优化**：为企业级操作设计高效的工作流

### 🏢 **企业工作流应用**
- **业务流程自动化**：自动化复杂的组织工作流
- **内容生产管道**：包含审查和批准阶段的编辑工作流
- **客户服务自动化**：多步骤客户查询解决方案
- **数据处理工作流**：具有 AI 驱动转换的 ETL 工作流

## ⚙️ 前提条件与设置

### 📦 **所需 NuGet 包**

此工作流演示使用了几个关键的 .NET 包：

```xml
<!-- Core AI Framework -->
<PackageReference Include="Microsoft.Extensions.AI" Version="9.9.0" />

<!-- Agent Framework (Local Development) -->
<!-- Microsoft.Agents.AI.dll - Core agent abstractions -->
<!-- Microsoft.Agents.AI.OpenAI.dll - OpenAI/GitHub Models integration -->

<!-- Configuration and Environment -->
<PackageReference Include="DotNetEnv" Version="3.1.1" />
```

### 🔑 **GitHub 模型配置**

**环境设置 (.env 文件)：**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

**GitHub 模型访问：**
1. 注册 GitHub 模型（目前处于预览阶段）
2. 生成具有模型访问权限的个人访问令牌
3. 按上述方式配置环境变量

### 🏗️ **工作流架构概述**

```mermaid
graph TD
    A[Workflow Builder] --> B[Agent Registry]
    B --> C[Workflow Execution Engine]
    C --> D[Agent 1: Content Generator]
    C --> E[Agent 2: Content Reviewer] 
    D --> F[Workflow Results]
    E --> F
    G[GitHub Models API] --> D
    G --> E
```

**关键组件：**
- **WorkflowBuilder**：设计工作流的主要编排引擎
- **AIAgent**：具有特定能力的单个专业代理
- **GitHub Models Client**：AI 模型推理服务集成
- **Execution Context**：管理工作流阶段之间的状态和数据流

## 🎨 **企业工作流设计模式**

### 📝 **内容生产工作流**
```
User Request → Content Generation → Quality Review → Final Output
```

### 🔍 **文档处理管道**
```
Document Input → Analysis → Extraction → Validation → Structured Output
```

### 💼 **商业智能工作流**
```
Data Collection → Processing → Analysis → Report Generation → Distribution
```

### 🤝 **客户服务自动化**
```
Customer Inquiry → Classification → Processing → Response Generation → Follow-up
```

## 🏢 **企业优势**

### 🎯 **可靠性与可扩展性**
- **确定性执行**：一致、可重复的工作流结果
- **错误恢复**：在任何工作流阶段优雅地处理故障
- **性能监控**：跟踪执行指标和优化机会
- **资源管理**：高效分配和利用 AI 模型资源

### 🔒 **安全性与合规性**
- **安全认证**：基于 GitHub 令牌的 API 访问认证
- **审计记录**：完整记录工作流执行和决策点
- **访问控制**：对工作流执行和监控的细粒度权限
- **数据隐私**：在整个工作流中安全处理敏感信息

### 📊 **可观察性与管理**
- **可视化工作流设计**：清晰表示流程流和依赖关系
- **执行监控**：实时跟踪工作流进度和性能
- **错误报告**：详细的错误分析和调试能力
- **性能分析**：优化和容量规划的指标

让我们开始构建您的第一个企业级 AI 工作流吧！🚀


In [1]:
#r "nuget: Microsoft.Extensions.AI, 9.9.1"

Installed Packages Microsoft.Extensions.AI, 9.9.1

In [2]:
#r "nuget: System.ClientModel, 1.6.1.0"

Installed Packages System.ClientModel, 1.6.1

In [3]:
#r "nuget: Azure.Identity, 1.15.0"
#r "nuget: System.Linq.Async, 6.0.3"
#r "nuget: OpenTelemetry.Api, 1.0.0"
#r "nuget: OpenTelemetry.Api, 1.0.0"

Installed Packages Azure.Identity, 1.15.0 OpenTelemetry.Api, 1.0.1 System.Linq.Async, 6.0.3

In [5]:

#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.Workflows, 1.0.0-preview.251001.3

In [ ]:

#r "nuget: Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.3"

Installed Packages Microsoft.Agents.AI.OpenAI, 1.0.0-preview.251001.2

In [7]:
#r "nuget: DotNetEnv, 3.1.1"

Installed Packages DotNetEnv, 3.1.1

In [8]:
// #r "nuget: Microsoft.Extensions.AI.OpenAI, 9.9.0-preview.1.25458.4"

In [9]:
using System;
using System.ComponentModel;
using System.ClientModel;
using OpenAI;
using Azure.Identity;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;

In [10]:
 using DotNetEnv;

In [11]:
Env.Load("../../../.env");

In [12]:

var github_endpoint = Environment.GetEnvironmentVariable("GITHUB_ENDPOINT") ?? throw new InvalidOperationException("GITHUB_ENDPOINT is not set.");
var github_model_id = Environment.GetEnvironmentVariable("GITHUB_MODEL_ID") ?? "gpt-4o-mini";
var github_token = Environment.GetEnvironmentVariable("GITHUB_TOKEN") ?? throw new InvalidOperationException("GITHUB_TOKEN is not set.");

In [13]:
var openAIOptions = new OpenAIClientOptions()
{
    Endpoint = new Uri(github_endpoint)
};

In [14]:
var openAIClient = new OpenAIClient(new ApiKeyCredential(github_token), openAIOptions);

In [15]:
const string ReviewerAgentName = "Concierge";
const string ReviewerAgentInstructions = @"
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. ";

In [16]:
const string FrontDeskAgentName = "FrontDesk";
const string FrontDeskAgentInstructions = @"""
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """;

In [17]:
AIAgent reviewerAgent = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:ReviewerAgentName,instructions:ReviewerAgentInstructions);
AIAgent frontDeskAgent  = openAIClient.GetChatClient(github_model_id).CreateAIAgent(
    name:FrontDeskAgentName,instructions:FrontDeskAgentInstructions);

In [18]:
var workflow = new WorkflowBuilder(frontDeskAgent)
            .AddEdge(frontDeskAgent, reviewerAgent)
            .Build();

In [19]:
ChatMessage userMessage = new ChatMessage(ChatRole.User, [
	new TextContent("I would like to go to Paris.") 
]);

In [20]:
StreamingRun run = await InProcessExecution.StreamAsync(workflow, userMessage);

In [21]:
await run.TrySendMessageAsync(new TurnToken(emitEvents: true));
string id="";
string messageData="";
await foreach (WorkflowEvent evt in run.WatchStreamAsync().ConfigureAwait(false))
{
    if (evt is AgentRunUpdateEvent executorComplete)
    {
        if(id=="")
        {
            id=executorComplete.ExecutorId;
        }
        if(id==executorComplete.ExecutorId)
        {
            messageData+=executorComplete.Data.ToString();
        }
        else
        {
            id=executorComplete.ExecutorId;
        }
        // Console.WriteLine($"{executorComplete.ExecutorId}: {executorComplete.Data}");
    }
}

Console.WriteLine(messageData);

Visit the Louvre Museum. It's a must-see for art enthusiasts and history lovers.That recommendation is quite popular and likely to attract many tourists. To refine it for a more local and authentic experience, consider suggesting an alternative that focuses on smaller, lesser-known art venues or galleries. Look for places where local artists exhibit or community spaces that host cultural events. This approach allows travelers to connect with the local art scene more intimately, away from the typical tourist routes.



---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
